# Week 1 - Data Wrangling

### import packages

In [1]:
#Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly as py
import datetime as dt
import scipy as scp
from scipy import stats
from scipy.stats import ttest_ind

import warnings
warnings.filterwarnings('ignore')

### create dataFrames

In [2]:
applications = pd.read_csv('../Data/application_record.csv')

In [3]:
records = pd.read_csv('../Data/credit_record.csv')

In [4]:
applications.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


#### it appears that there may be some duplicate records. deal with this later.

In [5]:
# check for missing values
applications.isna().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [6]:
# subset data - drop variables that most likely won't affect the DV.
applications2 = applications.drop(['FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL'], axis=1)

In [7]:
# rename columns 
applications2.rename(columns={'CODE_GENDER': 'gender', 'FLAG_OWN_CAR' : 'ownsCar', 'FLAG_OWN_REALTY' : 'ownsRealty', 'CNT_CHILDREN' : 'numChildren', 'AMT_INCOME_TOTAL' : 'totalIncome', 'NAME_INCOME_TYPE' : 'incomeType', 'NAME_EDUCATION_TYPE' : 'eduLvl', 'NAME_FAMILY_STATUS': 'famStatus', 'NAME_HOUSING_TYPE' : 'housingType', 'DAYS_BIRTH' : 'daysBirth', 'DAYS_EMPLOYED' : 'daysEmpl', 'OCCUPATION_TYPE' : 'occupation', 'CNT_FAM_MEMBERS' : 'famSize'}, inplace = True)

In [8]:
# replace missing values for occupation type with "Not identified"
applications2['occupation'].fillna(value='Not identified', inplace=True)

In [9]:
# convert daysBirth to Age in years for clarity
applications2['ageYrs'] = np.ceil(pd.to_timedelta(applications2['daysBirth'], unit='D').dt.days / -365.25)
applications2.drop('daysBirth', axis = 1, inplace = True)

In [10]:
# convert daysEmpl to yrsEmpl to keep in the same units
applications2['yrsEmpl']=-applications2['daysEmpl']/365.2425
applications2.loc[applications2['yrsEmpl']<0,'yrsEmpl']=0
applications2.drop('daysEmpl', axis=1, inplace=True)

In [11]:
# drop duplicates
applications3=applications2.drop_duplicates(subset=applications2.columns[1:], keep='first')

In [12]:
applications3.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,occupation,famSize,ageYrs,yrsEmpl
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,Security staff,2.0,59.0,3.104787
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,1.0,53.0,8.353354
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,Not identified,1.0,62.0,0.000000
10,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,Accountants,2.0,47.0,2.105450


In [13]:
# make sure continuous data is not too related (>.9)
applications3.corr()

,ID,numChildren,totalIncome,famSize,ageYrs,yrsEmpl
ID,1.000000,0.010966,0.006415,0.009868,-0.040180,0.016400
numChildren,0.010966,1.000000,0.003858,0.885066,-0.292065,-0.000498
totalIncome,0.006415,0.003858,1.000000,0.004004,-0.003904,0.052824
famSize,0.009868,0.885066,0.004004,1.000000,-0.244547,0.033949
ageYrs,-0.040180,-0.292065,-0.003904,-0.244547,1.000000,0.105017
yrsEmpl,0.016400,-0.000498,0.052824,0.033949,0.105017,1.000000


In [14]:
# This data appears to be independent, which is great for our analyses.

In [18]:
applications3.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,occupation,famSize,ageYrs,yrsEmpl,genderR,ownsCarR,ownsRealtyR
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,Security staff,2.0,59.0,3.104787,1,1,1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,1.0,53.0,8.353354,0,0,1
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,Not identified,1.0,62.0,0.000000,0,0,1
10,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,Accountants,2.0,47.0,2.105450,1,1,1


In [19]:
# Check out what we have for records
records.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [20]:
# look at unique status types for applicant's records
records['STATUS'].unique()

array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object)

In [21]:
records.STATUS.value_counts()

C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: STATUS, dtype: int64

#### From source explanation: C= account paid (up-to-date), X= 0 balance for the month, 0= payment overdue 1-29 days, 1= payment overdue 1-29 days, 2= payment overdue 60-89 days, 3= payment overdue 90-119 days, 4= overdue 120-149 days, 5= overdue >150 days. We will call anything more than 60 days payment past due delinquent/bad. 

In [22]:
# find unique months balance values
records['MONTHS_BALANCE'].unique()

array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
       -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
       -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
       -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
       -52, -53, -54, -55, -56, -57, -58, -59, -60], dtype=int64)

In [45]:
# recode status types to account for 60 day cutoff. 
def status(s):
    if s == "C":
        return 0
    elif s == "X":
        return 0
    elif s == '0':
        return 0
    elif s == '1':
        return 0
    elif s == '2':
        return 1
    elif s == '3':
        return 1
    elif s == '4':
        return 1
    elif s == '5':
        return 1

In [42]:
records['STATUS'] = records['STATUS'].apply(lambda x:status(x))

#### Define approval/rejection terms: If the number of months where applicant's account is up-to-date (<59 days) is higher than the number of months in 'default', the applicant gets the approval.


In [46]:
APPROVAL_STATUS = []
records.loc[(records[0] > records[1]), 'APPROVAL_STATUS'] = 1
records.loc[(records[0] <= records[1]), 'APPROVAL_STATUS'] = 0

KeyError: 0

In [25]:
records.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,0
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,0


# Data Visualization (placeholder until week 5)

In [26]:
# merge the datasets
merged = pd.merge(applications3, records, on="ID")

In [27]:
merged.head()

,ID,gender,ownsCar,ownsRealty,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,occupation,famSize,ageYrs,yrsEmpl,genderR,ownsCarR,ownsRealtyR,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,0,0
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-1,0
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-2,0
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-3,0
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-4,0


In [19]:
# check data types of all variables
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212932 entries, 0 to 212931
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              212932 non-null  int64  
 1   gender          212932 non-null  object 
 2   ownsCar         212932 non-null  object 
 3   ownsRealty      212932 non-null  object 
 4   numChildren     212932 non-null  int64  
 5   totalIncome     212932 non-null  float64
 6   incomeType      212932 non-null  object 
 7   eduLvl          212932 non-null  object 
 8   famStatus       212932 non-null  object 
 9   housingType     212932 non-null  object 
 10  occupation      212932 non-null  object 
 11  famSize         212932 non-null  float64
 12  ageYrs          212932 non-null  float64
 13  yrsEmpl         212932 non-null  float64
 14  MONTHS_BALANCE  212932 non-null  int64  
 15  STATUS          212932 non-null  object 
dtypes: float64(4), int64(3), object(9)
memory usage: 27.6+ M

In [40]:
#ID column is no longer necessary, so it is dropped
merged.drop(['ID'], axis=1, inplace=True)

In [22]:
# convert continuous variables to integers
merged.numChildren = merged.numChildren.astype(int)
merged.totalIncome = merged.totalIncome.astype(int)
merged.ageYrs = merged.ageYrs.astype(int)
merged.yrsEmpl = merged.yrsEmpl.astype(int)
merged.famSize = merged.famSize.astype(int)

In [23]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212932 entries, 0 to 212931
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              212932 non-null  int64 
 1   gender          212932 non-null  object
 2   ownsCar         212932 non-null  object
 3   ownsRealty      212932 non-null  object
 4   numChildren     212932 non-null  int32 
 5   totalIncome     212932 non-null  int32 
 6   incomeType      212932 non-null  object
 7   eduLvl          212932 non-null  object
 8   famStatus       212932 non-null  object
 9   housingType     212932 non-null  object
 10  occupation      212932 non-null  object
 11  famSize         212932 non-null  int32 
 12  ageYrs          212932 non-null  int32 
 13  yrsEmpl         212932 non-null  int32 
 14  MONTHS_BALANCE  212932 non-null  int64 
 15  STATUS          212932 non-null  object
dtypes: int32(5), int64(2), object(9)
memory usage: 23.6+ MB


### Recode String Variables

In [32]:
# recode, drop original string columns, and ensure int data type
def genderRecode (series):
    if series == "F":
        return 0
    if series == "M":
        return 1
merged['genderR'] = merged['gender'].apply(genderRecode)
merged.drop('gender', axis=1, inplace=True)

In [33]:
merged.genderR = merged.genderR.astype(int)

In [34]:
def carRecode (series):
    if series == "N":
        return 0
    if series == "Y":
        return 1
merged['ownsCarR'] = merged['ownsCar'].apply(carRecode)
merged.drop('ownsCar', axis=1, inplace=True)

In [35]:
merged.ownsCarR = merged.ownsCarR.astype(int)

In [36]:
def realtyRecode (series):
    if series == "N":
        return 0
    if series == "Y":
        return 1
merged['ownsRealtyR'] = merged['ownsRealty'].apply(realtyRecode)
merged.drop('ownsRealty', axis=1, inplace=True)

In [37]:
merged.ownsRealtyR = merged.ownsRealtyR.astype(int)

In [47]:
merged['occupation'].unique()

array(['Not identified', 'Security staff', 'Sales staff', 'Accountants',
       'Laborers', 'Managers', 'Drivers', 'Core staff',
       'High skill tech staff', 'Cleaning staff', 'Private service staff',
       'Cooking staff', 'Low-skill Laborers', 'Medicine staff',
       'Secretaries', 'Waiters/barmen staff', 'HR staff', 'Realty agents',
       'IT staff'], dtype=object)

In [51]:
def occuR (series):
    if series == "Not identified":
        return 0
    if series == "Security staff":
        return 1
    if series == "Sales staff":
        return 2
    if series == "Accountants":
        return 3
    if series == "Laborers":
        return 4
    if series == "Managers":
        return 5
    if series == "Drivers":
        return 6
    if series == "Core staff":
        return 7
    if series == "High skill tech staff":
        return 8
    if series == "Cleaning staff":
        return 9
    if series == "Private service staff":
        return 10
    if series == "Cooking staff":
        return 11
    if series == "Low-skill Laborers":
        return 12
    if series == "Medicine staff":
        return 13
    if series == "Secretaries":
        return 14
    if series == "Waiters/barmen staff":
        return 15
    if series == "HR staff":
        return 16
    if series == "Realty agents":
        return 17
    if series == "IT staff":
        return 18
    
merged['occupationR'] = merged['occupation'].apply(occuR)

In [55]:
merged['eduLvl'].unique()

array(['Higher education', 'Secondary / secondary special',
       'Incomplete higher', 'Lower secondary', 'Academic degree'],
      dtype=object)

In [56]:
def eduR (series):
    if series == "Higher education":
        return 0
    if series == "Secondary / secondary special":
        return 1
    if series == "Incomplete higher":
        return 2
    if series == "Lower secondary":
        return 3
    if series == "Academic degree":
        return 4
    
merged['eduLvlR'] = merged['eduLvl'].apply(eduR)

In [57]:
merged['incomeType'].unique()

array(['Working', 'Commercial associate', 'Pensioner', 'State servant',
       'Student'], dtype=object)

In [58]:
def incR (series):
    if series == "Working":
        return 0
    if series == "Commercial associate":
        return 1
    if series == "Pensioner":
        return 2
    if series == "State servant":
        return 3
    if series == "Student":
        return 4
    
merged['incomeTypeR'] = merged['incomeType'].apply(incR)

In [59]:
merged['housingType'].unique()

array(['Rented apartment', 'House / apartment', 'Municipal apartment',
       'With parents', 'Co-op apartment', 'Office apartment'],
      dtype=object)

In [60]:
def housR (series):
    if series == "Rented apartment":
        return 0
    if series == "House / apartment":
        return 1
    if series == "Municipal apartment":
        return 2
    if series == "With parents":
        return 3
    if series == "Co-op apartment":
        return 4
    if series == "Office apartment":
        return 5
    
merged['housingTypeR'] = merged['housingType'].apply(housR)

In [61]:
merged['famStatus'].unique()

array(['Civil marriage', 'Married', 'Single / not married', 'Separated',
       'Widow'], dtype=object)

In [62]:
def famR (series):
    if series == "Civil marriage":
        return 0
    if series == "Married":
        return 1
    if series == "Single / not married":
        return 2
    if series == "Separated":
        return 3
    if series == "Widow":
        return 4
    
merged['famStatusR'] = merged['famStatus'].apply(famR)

In [63]:
merged.head()

,numChildren,totalIncome,incomeType,eduLvl,famStatus,housingType,occupation,famSize,ageYrs,yrsEmpl,genderR,ownsCarR,ownsRealtyR,MONTHS_BALANCE,STATUS,occupationR,eduLvlR,incomeTypeR,housingTypeR,famStatusR
0,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,0,0,0,0,0,0,0
1,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-1,0,0,0,0,0,0
2,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-2,0,0,0,0,0,0
3,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-3,0,0,0,0,0,0
4,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Not identified,2.0,33.0,12.435574,1,1,1,-4,0,0,0,0,0,0


In [64]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212932 entries, 0 to 212931
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   numChildren     212932 non-null  int64  
 1   totalIncome     212932 non-null  float64
 2   incomeType      212932 non-null  object 
 3   eduLvl          212932 non-null  object 
 4   famStatus       212932 non-null  object 
 5   housingType     212932 non-null  object 
 6   occupation      212932 non-null  object 
 7   famSize         212932 non-null  float64
 8   ageYrs          212932 non-null  float64
 9   yrsEmpl         212932 non-null  float64
 10  genderR         212932 non-null  int32  
 11  ownsCarR        212932 non-null  int32  
 12  ownsRealtyR     212932 non-null  int32  
 13  MONTHS_BALANCE  212932 non-null  int64  
 14  STATUS          212932 non-null  int64  
 15  occupationR     212932 non-null  int64  
 16  eduLvlR         212932 non-null  int64  
 17  incomeType